# LSTM for Trump text generation

![alt text](https://miro.medium.com/v2/resize:fit:984/1*Mb_L_slY9rjMr8-IADHvwg.png)

Tensorflow will be used for the integration of the LSTM model.

## Dataset:

In [1]:
import os

path = 'Trump Rally Speeches/'
files = os.listdir(path)
files = [path + file for file in files]
 
dates = []
locations = []
years = []
days = []
months = []
speeches_text = []
 
month_ab = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct', 'Nov', 'Dec']

for file in files:
    for month in month_ab:
        if month in file:
            locations.append(file[file.find('/')+1:file.find(month)])
            break
    for i, mont in enumerate(month_ab):
        if month in file:
            date = file[file.find(month):file.find('.txt')]
            dates.append(date)
            months.append(date[:3])
            days.append(str(date[3]))
            years.append(date[-4:])
            break   
        
for file in files:
    with open(file, 'r') as f:
        speeches_text.append(f.read())     
        
import pandas as pd
 
df = pd.DataFrame({'Speech':files, 'Date':dates, 'Location':locations, 'Year':years, 'Month':months, 'Day':days, 'Speech_Text':speeches_text})

We will use the two preprocessing function defined in preprocessing_pipline.
One only remove the punctuation and do lower case, the other one also remove the stop and rare words.

In [2]:
from preprocessing import preprocessing_pipline

preprocessing = preprocessing_pipline(df['Speech_Text'])
df['Speech_Text_prepro'] = preprocessing.preprocess_light()
df['Speech_Text_prepro2'] = preprocessing.preprocess()


thank thank thank vice president pence hes good guy weve done great job together merry christmas mic
Thank you. Thank you. Thank you to Vice President Pence. He's a good guy. We've done a great job tog
thank thank thank vice president pence hes good guy weve done great job together merry christmas mic


### Data 
1. Without preprocessing -> terrible results... not intresting
2. Light preprocessing 
3. Heavy preprocessing (stop words and rare words)

In [21]:
from sklearn.model_selection import train_test_split

text_corpus = [word for speech in df['Speech_Text'].str.split() for word in speech]

# Preprocess text 
text_corpus_prepro = [word for speech in df['Speech_Text_prepro'].str.split() for word in speech]

text_corpus_prepro2 = [word for speech in df['Speech_Text_prepro2'].str.split() for word in speech]


In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

def preprocess_for_LSTM(text_corpus, lenght_of_sequences=10, test_size=0.2):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text_corpus)

    input = []
    output = []


    for i in range(lenght_of_sequences, len(text_corpus)):
        input.append(text_corpus[i - lenght_of_sequences:i])
        output.append(text_corpus[i])

    input = tokenizer.texts_to_sequences(input)
    output = tokenizer.texts_to_sequences(output)

    # pad 
    input = pad_sequences(input, maxlen=lenght_of_sequences, padding='pre')
    output = pad_sequences(output, maxlen=1, padding='pre')
    # Convert your lists of input sequences into numpy arrays
    
    input = np.array(input)
    output = np.array(output)

    # Split your data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(input, output, test_size=test_size, random_state=42)
    
    return X_train, X_test, y_train, y_test, tokenizer

 

In [23]:
X_train, X_test, y_train, y_test, tokenizer = preprocess_for_LSTM(text_corpus_prepro, max_sequence_length=10, test_size=0.2)
X_train2, X_test2, y_train2, y_test2, tokenizer2 = preprocess_for_LSTM(text_corpus_prepro2, max_sequence_length=10, test_size=0.2)


## The model

In [6]:
import tensorflow as tf
from keras.layers import Embedding, LSTM, Dense

class my_model_LSTM:
    def __init__(self, n, num_unique_words, max_sequence_length):
        self.n = n
        self.num_unique_words = num_unique_words
        self.max_sequence_length = max_sequence_length
        self.model = self.build_model()
        
    def build_model(self):
        model = tf.keras.Sequential()
        model.add(Embedding(self.num_unique_words, self.n))
        model.add(LSTM(units=self.n))
        model.add(Dense(units=self.num_unique_words, activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
     
    def train(self, X_train, y_train, epochs=10):
        self.model.fit(X_train, y_train, epochs=epochs)
         
    def predict(self, test_corpus):
        return self.model.predict(test_corpus)
     
    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)
     
    def save(self, path):
        self.model.save(path)
         
    def load(self, path):
        self.model = tf.keras.models.load_model(path)
         
    def generate_text(self, seed_text, max_length=100):
        output_text = seed_text.split()
        prefix = output_text[-(self.n - 1):]

        for _ in range(max_length):
            input_eval = [tokenizer.word_index[word] for word in prefix]
            input_eval = tf.expand_dims(input_eval, 0)

            predictions = self.model(input_eval)
            predictions = tf.squeeze(predictions, 0)

            if len(predictions.shape) == 1:
                predictions = tf.expand_dims(predictions, 0)

            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
            next_word = tokenizer.index_word[predicted_id]

            output_text.append(next_word)
            prefix = prefix[1:] + [next_word]

        return ' '.join(output_text)



### Light prepro

In [30]:
num_unique_words = len(tokenizer.word_index) + 1
max_sequence_length = 40
model_LSTM = my_model_LSTM(50, num_unique_words, max_sequence_length)
model_LSTM.train(X_train, y_train, epochs=5)

Epoch 1/5
4720/4720 [==============================] - 59s 12ms/step - loss: 7.3409 - accuracy: 0.0137
Epoch 2/5
4720/4720 [==============================] - 58s 12ms/step - loss: 6.8646 - accuracy: 0.0350
Epoch 3/5
4720/4720 [==============================] - 58s 12ms/step - loss: 6.4463 - accuracy: 0.0673
Epoch 4/5
4720/4720 [==============================] - 58s 12ms/step - loss: 6.0864 - accuracy: 0.0978
Epoch 5/5
4720/4720 [==============================] - 58s 12ms/step - loss: 5.7856 - accuracy: 0.1230


In [31]:
model_LSTM.save('model_LSTM.h5')

In [32]:
predictions = model_LSTM.evaluate(X_test, y_test)
print("LSTM Model Accuracy:", predictions[1])
print("LSTM Model Loss:", predictions[0])

1180/1180 [==============================] - 6s 5ms/step - loss: 6.3623 - accuracy: 0.1160
LSTM Model Accuracy: 0.11599883437156677
LSTM Model Loss: 6.362343788146973


In [33]:
model_LSTM = my_model_LSTM(256, num_unique_words, max_sequence_length)
model_LSTM.load('model_LSTM.h5')
seed_text = 'france forcing'
generated_text = model_LSTM.generate_text(seed_text, max_length=10)
print(generated_text)

france forcing prosperous patchwork mugging whole… toughest rational complain develop guy… weaken


dfdf

In [34]:
num_unique_words2 = len(tokenizer2.word_index) + 1
max_sequence_length = 40
model_LSTM2 = my_model_LSTM(50, num_unique_words2, max_sequence_length)
model_LSTM2.train(X_train2, y_train2, epochs=5)

Epoch 1/5
4720/4720 [==============================] - 59s 12ms/step - loss: 7.3365 - accuracy: 0.0145
Epoch 2/5
4720/4720 [==============================] - 58s 12ms/step - loss: 6.8391 - accuracy: 0.0362
Epoch 3/5
4720/4720 [==============================] - 58s 12ms/step - loss: 6.4185 - accuracy: 0.0691
Epoch 4/5
4720/4720 [==============================] - 58s 12ms/step - loss: 6.0564 - accuracy: 0.0999
Epoch 5/5
4720/4720 [==============================] - 58s 12ms/step - loss: 5.7570 - accuracy: 0.1239


In [36]:
model_LSTM2.save('model_LSTM2.h5')

In [37]:
predictions = model_LSTM2.evaluate(X_test, y_test)
print("LSTM Model Accuracy:", predictions[1])
print("LSTM Model Loss:", predictions[0])

1180/1180 [==============================] - 6s 5ms/step - loss: 6.3511 - accuracy: 0.1171
LSTM Model Accuracy: 0.11713763326406479
LSTM Model Loss: 6.35109281539917


In [38]:
model_LSTM2 = my_model_LSTM(256, num_unique_words, max_sequence_length)
model_LSTM2.load('model_LSTM.h5')
seed_text = 'hello'
generated_text = model_LSTM2.generate_text(seed_text, max_length=10)
print(generated_text)

hello speed onetenth oceans except going pardon garden iraq them… dousing


## Model